# Testing ChatESG

In [1]:
import requests
import json

In [5]:
TEST_UID = "#test-uid"
MODE = "local"

if MODE=="local":
    base_url = 'http://localhost:8000'

# Health check
requests.get(f"{base_url}/health").content, base_url

(b'{"status":"OK"}', 'http://localhost:8000')

### Chat

In [22]:
headers_chat = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

chat_payload = {
    "src": "skripsi",
    "model": "gpt-4o-mini",
    "bsid": "c23aabb84ff24825be7d",
    "inputs": [
        # {"type": "text", "role": "user", "content": "hai apa itu esg, jelasin secara singkat"}
        # {"type": "text", "role": "user", "content": "sebutkan 5 jenis inisiatif esg"}
        {"type": "text", "role": "user", "content": "rangkum percakapan"}
    ]
}

response = requests.post(f"{base_url}/api/v1/chat", headers=headers_chat, data=json.dumps(chat_payload), stream=True)
for line in response.iter_lines():
    if line:
        chunk = json.loads(line.decode('utf-8'))
        print("Chunk received:", chunk)
chunk

Chunk received: {'done': False, 'tools': ['Respon diterima'], 'responses': [], 'sources': None, 'bsid': 'c23aabb84ff24825be7d', 'bcid': '5c11a695-c5a6-4412-8db1-18dc6046b892', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Respon diterima', 'Mengambil histori percakapan'], 'responses': [], 'sources': None, 'bsid': 'c23aabb84ff24825be7d', 'bcid': '5c11a695-c5a6-4412-8db1-18dc6046b892', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Respon diterima', 'Mengambil histori percakapan'], 'responses': [''], 'sources': None, 'bsid': 'c23aabb84ff24825be7d', 'bcid': '5c11a695-c5a6-4412-8db1-18dc6046b892', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Respon diterima', 'Mengambil histori percakapan'], 'responses': ['Perc'], 'sources': None, 'bsid': 'c23aabb84ff24825be7d', 'bcid': '5c11a695-c5a6-4412-8db1-18dc6046b892', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Respon diterima', 'Mengambil histori percakapan'], 'responses': ['Percak'], '

{'done': True,
 'tools': ['Respon diterima', 'Mengambil histori percakapan'],
 'responses': ['Percakapan ini berfokus pada aspirasi keberlanjutan PT Astra International Tbk, yang dikenal sebagai Astra 2030 Sustainability Aspirations. Berikut adalah ringkasan dari percakapan:\n\n1. **Aspirasi Keberlanjutan**: Astra telah menetapkan 10 aspirasi dan target keberlanjutan untuk dicapai pada tahun 2030, yang mencakup pertumbuhan berkelanjutan dan dampak positif terhadap masyarakat dan lingkungan.\n\n2. **Progres dan Tantangan**: Pada tahun 2023, Astra melaporkan kemajuan yang memuaskan dalam mencapai aspirasi tersebut, meskipun masih ada tantangan yang harus dihadapi untuk memastikan relevansi dan keberlanjutan di masa depan.\n\n3. **Kolaborasi dan Peningkatan Kapasitas**: Astra menekankan pentingnya kerja sama, berbagi pengalaman, dan peningkatan kapasitas dalam mencapai tujuan keberlanjutan.\n\n4. **Kontribusi terhadap SDGs**: Astra berkomitmen untuk berkontribusi pada pencapaian Sustainab